<a href="https://colab.research.google.com/github/AshwinDeshpande96/LANL-Earthquake-Prediction/blob/master/LANL_Earthquake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Raw Data Fetching

In [7]:
#Get 'kaggle.json' file from your kaggle profile. Click on 'Edit Profile' --> 'Create New API Token'
#Upload 'kaggle.json' file
from google.colab import files
files.upload()

{}

In [6]:
#Make /root/.kaggle/ directory. So that kaggle command can download the data
!mkdir /root/.kaggle/
#Move downloaded 'kaggle.json' to previously created directory
!mv /content/kaggle.json /root/.kaggle/
#change permissions of 'kaggle.json' so that it is only accessible by you
!chmod 600 /root/.kaggle/kaggle.json
!ls /content/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists
mv: cannot stat '/content/kaggle.json': No such file or directory
gdrive	sample_data  sample_submission.csv  test.zip  train.csv.zip


In [5]:
#download the Data given by Kaggle Competition
!kaggle competitions download -c LANL-Earthquake-Prediction
#this downloads 3 files: 
#1) sample_submisison.csv  
#2) test.zip - (this contains 2624 csv files with 150000 rows of acoustic_data)   
#3) train.csv.zip - Contains one csv file with approx 150000*4195 rows of ['acoustic_data', 'time_to_failure']
!ls

  0% 0.00/33.3k [00:00<?, ?B/s]
100% 33.3k/33.3k [00:00<00:00, 32.9MB/s]
 96% 233M/242M [00:02<00:00, 82.3MB/s]
100% 242M/242M [00:02<00:00, 98.7MB/s]
 99% 2.01G/2.03G [00:17<00:00, 90.9MB/s]
100% 2.03G/2.03G [00:17<00:00, 123MB/s] 


In [8]:
#extract train.csv.zip file to obtain train.csv file
!unzip train.csv.zip
!ls

Archive:  train.csv.zip
  inflating: train.csv               

In [0]:
#extract test.zip file to obtain 2624 'seg_******.csv' files
!unzip test.zip
!ls

# Feature Extraction

In [3]:
#Link up to Google Drive to store Extracted Feature Data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from sklearn.linear_model import LinearRegression
#this function returns a single feature: 'abs_trend'
def add_trend_feature(arr, abs_values=False):
    """Fit a univariate linear regression and return the coefficient."""
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]


In [0]:
#this function creates a row  124 features: ['abs_max', 'abs_mean', 'abs_std', 'abs_trend', 'ave', 'ave_roll_abs_mean_10', 'ave_roll_abs_mean_100', 'ave_roll_abs_mean_1000', 'ave_roll_abs_mean_50', 'ave_roll_mean_10', 
#'ave_roll_mean_100', 'ave_roll_mean_1000', 'ave_roll_mean_50', 'ave_roll_std_10', 'ave_roll_std_100', 'ave_roll_std_1000', 'ave_roll_std_50', 'max', 'max_roll_mean_10', 'max_roll_mean_100', 'max_roll_mean_1000', 
#'max_roll_mean_50', 'max_roll_min_10', 'max_roll_min_100', 'max_roll_min_1000', 'max_roll_min_50', 'max_roll_std_10', 'max_roll_std_100', 'max_roll_std_1000', 'max_roll_std_50', 'min', 'min_roll_max_10', 
#'min_roll_max_100', 'min_roll_max_1000', 'min_roll_max_50', 'min_roll_mean_10', 'min_roll_mean_100', 'min_roll_mean_1000', 'min_roll_mean_50', 'min_roll_std_10', 'min_roll_std_100', 'min_roll_std_1000', 
#'min_roll_std_50', 'q01', 'q01_roll_std_10', 'q01_roll_std_100', 'q01_roll_std_1000', 'q01_roll_std_50', 'q05', 'q05_roll_abs_mean_10', 'q05_roll_abs_mean_100', 'q05_roll_abs_mean_1000', 'q05_roll_abs_mean_50', 
#'q05_roll_max_10', 'q05_roll_max_100', 'q05_roll_max_1000', 'q05_roll_max_50', 'q05_roll_mean_10', 'q05_roll_mean_100', 'q05_roll_mean_1000', 'q05_roll_mean_50', 'q05_roll_min_10', 'q05_roll_min_100', 
#'q05_roll_min_1000', 'q05_roll_min_50', 'q05_roll_std_10', 'q05_roll_std_100', 'q05_roll_std_1000', 'q05_roll_std_50', 'q10', 'q10_roll_std_10', 'q10_roll_std_100', 'q10_roll_std_1000', 'q10_roll_std_50', 'q90', 'q95',
#'q95_roll_abs_mean_10', 'q95_roll_abs_mean_100', 'q95_roll_abs_mean_1000', 'q95_roll_abs_mean_50', 'q95_roll_max_10', 'q95_roll_max_100', 'q95_roll_max_1000', 'q95_roll_max_50', 'q95_roll_mean_10', 'q95_roll_mean_100',
#'q95_roll_mean_1000', 'q95_roll_mean_50', 'q95_roll_min_10', 'q95_roll_min_100', 'q95_roll_min_1000', 'q95_roll_min_50', 'q95_roll_std_10', 'q95_roll_std_100', 'q95_roll_std_1000', 'q95_roll_std_50', 'q99', 
#'q99_roll_std_10', 'q99_roll_std_100', 'q99_roll_std_1000', 'q99_roll_std_50', 'std', 'std_roll_abs_mean_10', 'std_roll_abs_mean_100', 'std_roll_abs_mean_1000', 'std_roll_abs_mean_50', 'std_roll_max_10', 
#'std_roll_max_100', 'std_roll_max_1000', 'std_roll_max_50', 'std_roll_mean_10', 'std_roll_mean_100', 'std_roll_mean_1000', 'std_roll_mean_50', 'std_roll_min_10', 'std_roll_min_100', 'std_roll_min_1000', 
#'std_roll_min_50', 'std_roll_std_10', 'std_roll_std_100', 'std_roll_std_1000', 'std_roll_std_50', 'std_to_mean', 'trend']
def extract_features_from_segment(x):
    """Returns a dictionary with the features for the given segment of acoustic data."""
    features = {}
    
    features['ave'] = x.values.mean()
    features['std'] = x.values.std()
    features['max'] = x.values.max()
    features['min'] = x.values.min()
    features['q90'] = np.quantile(x.values, 0.90)
    features['q95'] = np.quantile(x.values, 0.95)
    features['q99'] = np.quantile(x.values, 0.99)
    features['q05'] = np.quantile(x.values, 0.05)
    features['q10'] = np.quantile(x.values, 0.10)
    features['q01'] = np.quantile(x.values, 0.01)
    features['std_to_mean'] = features['std'] / features['ave']
    
    features['abs_max'] = np.abs(x.values).max()
    features['abs_mean'] = np.abs(x.values).mean()
    features['abs_std'] = np.abs(x.values).std()
    features['trend'] = add_trend_feature(x.values)
    features['abs_trend'] = add_trend_feature(x.values, abs_values=True)
    
    # New features - rolling features
    for w in [10, 50, 100, 1000]:
        x_roll_abs_mean = x.abs().rolling(w).mean().dropna().values
        x_roll_mean = x.rolling(w).mean().dropna().values
        x_roll_std = x.rolling(w).std().dropna().values
        x_roll_min = x.rolling(w).min().dropna().values
        x_roll_max = x.rolling(w).max().dropna().values
        
        features['ave_roll_std_' + str(w)] = x_roll_std.mean()
        features['std_roll_std_' + str(w)] = x_roll_std.std()
        features['max_roll_std_' + str(w)] = x_roll_std.max()
        features['min_roll_std_' + str(w)] = x_roll_std.min()
        features['q01_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.01)
        features['q05_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.05)
        features['q10_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.10)
        features['q95_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.95)
        features['q99_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.99)
        
        features['ave_roll_mean_' + str(w)] = x_roll_mean.mean()
        features['std_roll_mean_' + str(w)] = x_roll_mean.std()
        features['max_roll_mean_' + str(w)] = x_roll_mean.max()
        features['min_roll_mean_' + str(w)] = x_roll_mean.min()
        features['q05_roll_mean_' + str(w)] = np.quantile(x_roll_mean, 0.05)
        features['q95_roll_mean_' + str(w)] = np.quantile(x_roll_mean, 0.95)
        
        features['ave_roll_abs_mean_' + str(w)] = x_roll_abs_mean.mean()
        features['std_roll_abs_mean_' + str(w)] = x_roll_abs_mean.std()
        features['q05_roll_abs_mean_' + str(w)] = np.quantile(x_roll_abs_mean, 0.05)
        features['q95_roll_abs_mean_' + str(w)] = np.quantile(x_roll_abs_mean, 0.95)
        
        features['std_roll_min_' + str(w)] = x_roll_min.std()
        features['max_roll_min_' + str(w)] = x_roll_min.max()
        features['q05_roll_min_' + str(w)] = np.quantile(x_roll_min, 0.05)
        features['q95_roll_min_' + str(w)] = np.quantile(x_roll_min, 0.95)

        features['std_roll_max_' + str(w)] = x_roll_max.std()
        features['min_roll_max_' + str(w)] = x_roll_max.min()
        features['q05_roll_max_' + str(w)] = np.quantile(x_roll_max, 0.05)
        features['q95_roll_max_' + str(w)] = np.quantile(x_roll_max, 0.95)
    return features

In [0]:
import pandas as pd
import time as t
import os
#Feature Extraction found at https://www.kaggle.com/jsaguiar/baseline-with-multiple-models
#make_train function returns a 3 things:
#!) X_train - Training Set of shape (4195 * 124) - Each of 4195 rows is contructed by calling 'extract_features_from_segment' function from 150,000 data-chunk of train.csv file
#2) target - target variable is the value of every 150,000th  train.csv's ['time_to_failure'] data
#3) quake - not used in this project
def make_train():
    features_list = []
    target_list = []
    quake_num = []
    quake_count = 0
    row = 1
    time_so_far = 0
    for segment in pd.read_csv('/content/train.csv', chunksize=150000):
        
        start = t.time()
        
        target_list.append(segment['time_to_failure'].values[-1])
        features_list.append(extract_features_from_segment(segment['acoustic_data']))
        quake_num.append(quake_count)
        
        if any(segment['time_to_failure'].diff() > 5):
            quake_count = quake_count + 1
        
        end = t.time()
        
        time_taken = end - start
        time_so_far += time_taken
        
        print "\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b", row, "/4195 \n ETA: ", round(time_so_far/60, 1), "/", int(time_taken*4195/60) 
        
        row = row + 1
    return pd.DataFrame(features_list), pd.Series(target_list), pd.Series(quake_num)
#reads each 2624 'seg_******.csv' file and creates a (2624*124) dataset for testing
def make_test():
    features_list = {}
    
    row = 1
    time_so_far = 0
    test_files = [file for file in os.listdir(os.getcwd()) if file.startswith("seg")]          #returns an array of every seg_******.csv file in current directory
    num_test_file - len(test_files)
    for file in test_files:
        segment = pd.read_csv(file)
        start = t.time()
        
        
        features_list.append(extract_features_from_segment(segment['acoustic_data']))
        
        
        end = t.time()
        
        time_taken = end - start
        time_so_far += time_taken
        
        print "\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b", row, "\b/" +num_test_file+"\nETA: ", round(time_so_far/60, 1), "\b/", int(time_taken*num_test_file/60) 
        
        row = row + 1
    return pd.DataFrame(features_list)


In [0]:
#X_train, target, quake is created
X_train, target, quake = make_train()
print "Train shape:", X_train.shape, "\n", X_train.head(3), "\nTarget shape:", target.shape, "\n", target.head(3), "\nQuake shape:", quake.shape, "\n", quake.head(3), 


In [0]:
#files are saved onto Google Drive in csv format with X_train - (4195*124) rows & colums, target with (4195, 1) rows & columns, quake with (4195, 1) rows & columns
#change filepath according to your drive directory
X_train.to_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/X_train.csv", index=False)
target.to_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/target.csv", index=False)
quake.to_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/quake.csv", index=False)

In [63]:
#test_set is created
testset = make_test()
#files are saved onto Google Drive in csv format with test_set.csv - (2624*124) rows & colums
testset.to_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/test_set.csv", index=False)

 1 /4195 ETA:  0.0 / 15
 2 /4195 ETA:  0.0 / 15
 3 /4195 ETA:  0.0 / 15
 4 /4195 ETA:  0.0 / 15
 5 /4195 ETA:  0.0 / 15
 6 /4195 ETA:  0.0 / 15
 7 /4195 ETA:  0.0 / 15
 8 /4195 ETA:  0.0 / 15
 9 /4195 ETA:  0.0 / 15
 10 /4195 ETA:  0.0 / 15
 11 /4195 ETA:  0.0 / 15
 12 /4195 ETA:  0.0 / 16
 13 /4195 ETA:  0.0 / 15
 14 /4195 ETA:  0.1 / 15
 15 /4195 ETA:  0.1 / 15
 16 /4195 ETA:  0.1 / 15
 17 /4195 ETA:  0.1 / 15
 18 /419

Once X-train.csv, target.csv, quake.csv, test_set.csv are created we do not have to run previous cells again. 
Further program uses only the data saved in csv file in Google drive

# Data Import

In [0]:
#We start by importing previously created csv files
import pandas as pd
X = pd.read_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/X_train.csv")
Y = pd.read_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/target.csv")
#Uncomment if u tend to use quake data
#quake = pd.read_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/quake.csv")

In [117]:
#X_train.drop(X_train.tail(1).index,inplace=True)
print "Training Data:\t", X.shape
print "  Target Data:\t", Y.shape

#Display sample of training data
X.head(5)


Training Data:	(4194, 124)
  Target Data:	(4194, 1)


,abs_max,abs_mean,abs_std,abs_trend,ave,ave_roll_abs_mean_10,ave_roll_abs_mean_100,ave_roll_abs_mean_1000,ave_roll_abs_mean_50,ave_roll_mean_10,...,std_roll_min_10,std_roll_min_100,std_roll_min_1000,std_roll_min_50,std_roll_std_10,std_roll_std_100,std_roll_std_1000,std_roll_std_50,std_to_mean,trend
0,104,5.576567,4.333310,-0.000011,4.884113,5.576566,5.576655,5.579118,5.576561,4.884088,...,4.526280,6.522304,11.248606,5.787738,2.809071,3.111524,2.769772,3.176148,1.044425,-3.268300e-06
1,181,5.734167,5.732758,-0.000005,4.725767,5.734187,5.734653,5.739510,5.734444,4.725732,...,6.398675,10.162552,21.096014,8.823748,4.120785,4.893431,4.492905,4.953472,1.394229,9.090424e-07
2,140,6.152647,5.895925,0.000010,4.906393,6.152557,6.153045,6.159850,6.152678,4.906229,...,6.641086,10.535572,19.047827,9.125804,4.227960,4.959233,4.402140,5.035827,1.420060,3.962182e-06
3,199,5.933960,6.061193,-0.000002,4.902240,5.933982,5.934187,5.939037,5.933995,4.902291,...,6.695330,10.524292,20.248763,9.217610,4.284176,5.247130,4.873523,5.285204,1.412065,1.637207e-06
4,145,6.110587,6.329464,0.000005,4.908720,6.110644,6.111666,6.111178,6.111119,4.908728,...,7.153246,11.844273,20.321001,10.305776,4.183253,5.556924,5.213365,5.610383,1.487371,-6.668392e-07


#  Training

In [0]:
#import required libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [67]:
#split the training and target data into 3/4 for training and 1/4 for validation. Choose your split with test_size= values ranging from [0,1] percentage of data
X_train, X_test, y_train, y_test = train_test_split(X.values, Y.values, test_size=0.25, random_state=42)

(3145, 124)


# Pictorial representation of Neural Network used for Regression
![alt text](https://github.com/AshwinDeshpande96/LANL-Earthquake-Prediction/blob/master/regression%20model.jpeg?raw=true)

In [0]:
#We create a Fully Connected Dense Net Initial width 156 and Depth of 5 layers each activated by relu besides the last(output layer) as this is a regression type neural network
model = Sequential()
model.add(Dense(156 , input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(128,  kernel_initializer='normal', activation='relu'))
model.add(Dense(64,  kernel_initializer='normal', activation='relu'))
model.add(Dense(32,  kernel_initializer='normal', activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(metrics=['accuracy'],loss='logcosh', optimizer=Adam(lr=1.0e-3))

In [96]:
filepath = "/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/weights.hdf5"
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min'),                                   #Stop training if no improvement is seen in Validation loss after 10 attempts
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min'),                    #Slow down Learning rate if Validation Loss is not Decreasing after 5 attempts
    ModelCheckpoint(filepath, save_best_only=True,  save_weights_only=False, monitor='val_loss', mode='min') #Model weights are saved in hdf5 file every epoch
]
#model = load_model('/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/model.hdf5')
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, callbacks=callbacks, verbose=0) #Train Model
model.save('/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/model.hdf5')      #Save Trained model after training


Epoch 00006: ReduceLROnPlateau reducing learning rate to 1.00000011116e-07.

Epoch 00011: ReduceLROnPlateau reducing learning rate to 1.00000008274e-08.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 1.00000008274e-09.


# Testing

In [100]:
test_files = [file for file in os.listdir(os.getcwd()) if file.startswith("seg")]                                                       #Get all Test Segments Names for printing name and it's prediction
model = load_model('/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/model.hdf5')                         #Load Trained Model
test_df = pd.read_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/test_set.csv")                    #Read Test Data
print test_df.values.shape                                                                                                              #Check to see if 2624 Rows are present
ynew = model.predict(test_df.values)                                                                                                    #Predict Y value('time_to_failure') for ecery 2624 row
submission = []                                                                                                                         #Array to save Segment and it's prediction
for i in range(len(ynew)):
    submission.append( {"seg_id":test_files[i], "time_to_failure":ynew[i][0]})                                                          #Save every segment and it's prediction in submission array
pd.DataFrame(submission).to_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/submission.csv", index=False) #Save Predictions in a csv file

(2624, 124)


In [102]:
submission_df = pd.read_csv("/content/gdrive/My Drive/Projects/Kaggle Challenges/LANL Earthquake Prediction/submission.csv")           #Read Saved Submission File
print submission_df.head(5)                                                                                                            #Print first 5 predictions of 2624 rows

           seg_id  time_to_failure
0  seg_63c8e4.csv         9.291997
1  seg_5f352e.csv         2.569135
2  seg_8fc754.csv         5.890961
3  seg_f7290f.csv         2.364862
4  seg_2383c2.csv         4.226511


# Trials

In [0]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=model, epochs=100, batch_size=5, verbose=0)

In [0]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X_train, target, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [0]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X_train, target, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))